Building simple statistical model without usning any machine learning model

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from collections import OrderedDict
from datetime import datetime
import statistics as s
from pandas import datetime
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import warnings
import itertools

C:\Users\Rathore\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df = pd.read_csv("C:/Users/Rathore/Downloads/hotify/data/main_data")

In [3]:
df.head(50)

,cust_name,date_of_day,sales
0,Client 2035,9/24/2015,22.96
1,Client 2207,2/6/2016,26.25
2,Client 1094,2/20/2017,35.09
3,Client 1091,3/7/2016,22.29
4,Client 1999,12/27/2016,22.75
5,Client 945,9/13/2016,122.86
6,Client 2251,3/5/2016,2.04
7,Client 2251,5/12/2016,25.56
8,Client 1080,10/17/2016,50.62
9,Client 392,2/22/2017,78.16


In [5]:
'''# Data for plotting
for i in df.cust_name:
x = df[df['cust_name'] == 'i']['date_of_day']#.values[::-1][:50]
y = df[df['cust_name'] == 'i']['sales']#.values[::-1][:50]

x = x.tolist()
# x = sorted(x, key=lambda d: tuple(map(int, d.split('/'))), reverse=True)
print(x)

y = y.tolist()
print(y)
# y = y - y.mean 

temp_dict = dict()
for i, j in zip(x, y):
    if i in temp_dict:
        temp_dict[i] += j
    else:
        temp_dict[i] = j

x_sorted = sorted(temp_dict.keys(), key=lambda x: datetime.strptime(x, '%m/%d/%Y'))
print(x_sorted)

y_sorted = list()
for i in x_sorted:
    y_sorted.append(temp_dict[i])
    
print(y_sorted)
'''


"# Data for plotting\nfor i in df.cust_name:\nx = df[df['cust_name'] == 'i']['date_of_day']#.values[::-1][:50]\ny = df[df['cust_name'] == 'i']['sales']#.values[::-1][:50]\n\nx = x.tolist()\n# x = sorted(x, key=lambda d: tuple(map(int, d.split('/'))), reverse=True)\nprint(x)\n\ny = y.tolist()\nprint(y)\n# y = y - y.mean \n\ntemp_dict = dict()\nfor i, j in zip(x, y):\n    if i in temp_dict:\n        temp_dict[i] += j\n    else:\n        temp_dict[i] = j\n\nx_sorted = sorted(temp_dict.keys(), key=lambda x: datetime.strptime(x, '%m/%d/%Y'))\nprint(x_sorted)\n\ny_sorted = list()\nfor i in x_sorted:\n    y_sorted.append(temp_dict[i])\n    \nprint(y_sorted)\n"

Churning clients = clients whoes last five sales are in decreasing order

In [24]:
client_list = df['cust_name'].unique()
for client in client_list:

    # Unsorted 'date_of_day' and 'sales' data for a particular client
    x = df[df['cust_name'] == client]['date_of_day'].values[::-1][:10]
    y = df[df['cust_name'] == client]['sales'].values[::-1][:10]
    x_mean = df[df['cust_name'] == client]['sales']
    # Dictionary with dates as keys and sales as value
    temp_dict = dict()
    for i, j in zip(x, y):
        if i in temp_dict: temp_dict[i] += j
        else: temp_dict[i] = j

    # Sorted Dates
    x_sorted = sorted(temp_dict.keys(), key=lambda x: datetime.strptime(x, '%m/%d/%Y'))
    
    # Sorted Sales
    y_sorted = list()
    for i in x_sorted: y_sorted.append(temp_dict[i])
     
    # Get clients who have sales greater than 5 in number and last sale is negative
    if len(y_sorted) >= 5:
        last_five_sale = y_sorted[-5:]
        
        if last_five_sale == sorted(last_five_sale, reverse=True):
            print('{} is goint to quit!'.format(client))
    
    # break
    
    # print('---------------------------------------------')
print('Done!')
    

Client 1381 is goint to quit!
Client 946 is goint to quit!
Client 1660 is goint to quit!
Client 378 is goint to quit!
Client 823 is goint to quit!
Client 1692 is goint to quit!
Client 1276 is goint to quit!
Client 366 is goint to quit!
Client 452 is goint to quit!
Client 329 is goint to quit!
Done!


In [4]:
mean_sales = []
client_list = df['cust_name'].unique()
for client in client_list:

    # Unsorted 'date_of_day' and 'sales' data for a particular client
    x = df[df['cust_name'] == client]['date_of_day']
    y = df[df['cust_name'] == client]['sales']
    mean_sales.append(s.mean(y)) 
    


Churning clients whoes last five sales are in decreasing order and greatest among them is less than the mean sale of that client.

In [5]:
churning_client = list()
client_list = df['cust_name'].unique()
for client in client_list:

    # Unsorted 'date_of_day' and 'sales' data for a particular client
    x = df[df['cust_name'] == client]['date_of_day']
    y = df[df['cust_name'] == client]['sales']
    mean = s.mean(y)
    
    # Dictionary with dates as keys and sales as value
    temp_dict = dict()
    for i, j in zip(x, y):
        if i in temp_dict: temp_dict[i] += j
        else: temp_dict[i] = j

    # Sorted Dates
    x_sorted = sorted(temp_dict.keys(), key=lambda x: datetime.strptime(x, '%m/%d/%Y'))
    
    # Sorted Sales
    y_sorted = list()
    for i in x_sorted: y_sorted.append(temp_dict[i])
     
    # Get clients who have sales greater than 5 in number and last sale is negative
    if len(y_sorted) >= 5:
        last_five_sale = y_sorted[-5:]
        
        if last_five_sale[0] < mean:
            churning_client.append(client)
            print('{} is goint to quit!'.format(client))
    
    # break
    
    # print('---------------------------------------------')
print('Done!')
    

    

Client 2207 is goint to quit!
Client 1080 is goint to quit!
Client 2160 is goint to quit!
Client 2113 is goint to quit!
Client 2017 is goint to quit!
Client 1758 is goint to quit!
Client 1641 is goint to quit!
Client 1409 is goint to quit!
Client 1507 is goint to quit!
Client 1345 is goint to quit!
Client 133 is goint to quit!
Client 811 is goint to quit!
Client 1057 is goint to quit!
Client 356 is goint to quit!
Client 1135 is goint to quit!
Client 998 is goint to quit!
Client 1535 is goint to quit!
Client 864 is goint to quit!
Client 1291 is goint to quit!
Client 233 is goint to quit!
Client 155 is goint to quit!
Client 634 is goint to quit!
Client 409 is goint to quit!
Client 2022 is goint to quit!
Client 1730 is goint to quit!
Client 2134 is goint to quit!
Client 793 is goint to quit!
Client 922 is goint to quit!
Client 2234 is goint to quit!
Client 2224 is goint to quit!
Client 61 is goint to quit!
Client 1272 is goint to quit!
Client 1753 is goint to quit!
Client 4 is goint to qu

Client 1738 is goint to quit!
Done!


In [103]:
client_list

array(['Client 2035', 'Client 2207', 'Client 1094', ..., 'Client 1176',
       'Client 2059', 'Client 114'], dtype=object)